In [1]:
!pip install pandas
!pip install transformers
!pip install torch
!pip install nvidia-ml-py3
!pip install --no-cache-dir cloudpickle==2.2.1
!pip install mlflow==2.4

In [2]:
import pandas as pd
import torch 
import numpy as np
from transformers import BertTokenizerFast, BertForTokenClassification
from torch.utils.data import DataLoader
from tqdm import tqdm
from torch.optim import SGD

In [3]:
import mlflow
model = mlflow.pyfunc.load_model('models:/uc1_token/latest')

2023/07/18 02:25:42 WARNING mlflow.pyfunc: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - numpy (current: 1.21.6, required: numpy==1.21.2)
 - packaging (current: 23.0, required: packaging==23.1)
 - torch (current: 1.12.0, required: torch==2.0.1)
 - transformers (current: 4.16.0, required: transformers==4.30.0)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


[INFO] Model will use CPU runtime


In [8]:
sample =  pd.DataFrame({ 'text':["Gaok Men's Retro Cotton Multi-Pocket Camo Cargo Shorts"] })
model.predict(sample)

['B-BRAND',
 'O',
 'O',
 'O',
 'O',
 'B-PATTERN',
 'I-PATTERN',
 'I-PATTERN',
 'O',
 'O',
 'O',
 'O']

In [3]:
import ast
df = pd.read_csv('./data/output_fashion.csv', names=["text", "labels", "category"], skiprows=1)

# Drop the last column
df = df.drop(columns="category")
df = df.head(100)
# Function to convert the string representation to a list
def convert_to_list(string_representation):
    return ast.literal_eval(string_representation)

# Apply the function to the 'bio_tag' column for all rows
df['labels'] = df['labels'].apply(convert_to_list)


In [4]:
labels = [i for i in df['labels'].values.tolist()]
unique_labels = set()

for lb in labels:
    [unique_labels.add(i) for i in lb if i not in unique_labels]


# Sort the unique_labels
sorted_unique_labels = sorted(unique_labels)

# Create labels_to_ids and ids_to_labels mappings
labels_to_ids = {label: idx for idx, label in enumerate(sorted_unique_labels)}
ids_to_labels = {idx: label for idx, label in enumerate(sorted_unique_labels)}

In [5]:
# Function to convert the space-separated string to a list
def convert_to_list(string_representation):
    return " ".join(string_representation)

# Apply the function to the 'bio_tag' column for all rows
df['labels'] = df['labels'].apply(convert_to_list)

In [6]:
def extract_entities_from_bio(sentence, bio_tags):
    entities = {}
    current_entity = None
    current_entity_type = None
    words = sentence.split()
    tags = bio_tags.split()

    for word, tag in zip(words, tags):
        if tag.startswith("B-"):
            if current_entity:
                entity_value = " ".join(current_entity)
                entities[entity_value] = current_entity_type

            current_entity = [word]
            current_entity_type = tag[2:]
        elif tag.startswith("I-"):
            if current_entity:
                current_entity.append(word)
        else:
            if current_entity:
                entity_value = " ".join(current_entity)
                entities[entity_value] = current_entity_type
                current_entity = None
                current_entity_type = None

    if current_entity:
        entity_value = " ".join(current_entity)
        entities[entity_value] = current_entity_type

    return entities


In [7]:
import json
import pandas as pd
from collections import OrderedDict

# The extract_entities_from_bio function (keep it as it is)

# Given JSON input
json_input = '''
{
"data": [
		{
			"Title": "Gaok Men's Retro Cotton Multi-Pocket Camo Cargo Shorts",
			"Description": "Random Description",
			"Price": "13",
			"Category": "Jeany Jeans",
			"Image": "https://www.ssxsds.de/test.jpg"
		},
		{
			"Title": "Honchosfx Mens Retro Creedence Clearwater Revival 1971 T-Shirt Brown",
			"Description": "ssdsdsdssdssd",
			"Price": "15",
			"Category": "random",
			"Image": "https://www.ssxsds.de/test.jpg"
		}
	]
}
'''

# Load the JSON input
data = json.loads(json_input)

# Initialize the output_data with the given structure
output_data = {
    "status": "success",
    "result": [{
        "general": [{
            "types": ["Dog supplies"],
            "all_attributes": [{
                "Title": "string",
                "Description": "string",
                "Price": "decimal",
                "Category": "string",
                "Type": "string",
                "Image": "file"
            }],
            "common_attributes": [{
                "Title": "string",
                "Description": "string",
                "Price": "file",
                "Category": "string"
            }]
        }]
    }],
    "data": []
}

# Create the "tags" field for the "Category" key
tags_data = [{
    "attribute": "category",
    "probability": "100%"
}]

# Loop through each data item and process them individually
for item in data["data"]:
    # Create an ordered dictionary to maintain the key order for each data item
    category_output = OrderedDict()
    sample =  pd.DataFrame({ 'text':[item['Title']]})
    bio_tags = " ".join(model.predict(sample))
    # Extract entities for the "Title" attribute using the given function
    title_entities = extract_entities_from_bio(item["Title"], bio_tags)  # Assuming 'bio_tags' is defined earlier

    # Add the keys in the desired order for the current data item
    for key in ["Title", "Description", "Price", "Category", "Image"]:
        if key == "Category":
            category_output[key] = [
                {
                    "input": item[key],
                    "status": "mapped",
                    "tags": {
                        item[key]: tags_data
                    }
                }
            ]
        elif key == "Title":
            if title_entities:
                title_tags = []
                for entity_key, entity_value in title_entities.items():
                    if entity_key in item[key]:
                        title_tags.append({entity_key: [{"attribute": entity_value}]})
                category_output[key] = [
                    {
                        "input": item[key],
                        "status": "mapped",
                        "tags": title_tags
                    }
                ]
            else:
                category_output[key] = [
                    {
                        "input": item[key],
                        "status": "unmapped"
                    }
                ]
        else:
            category_output[key] = [
                {
                    "input": item[key],
                    "status": "unmapped"
                }
            ]

    # Add the output format for the current data item to the "data" list
    output_data["data"].append(category_output)

# Save the output data to a new JSON file
with open("output1.json", "w") as f:
    json.dump(output_data, f, indent=4)

print("Successfully created 'output3.json' with the desired output format.")


NameError: name 'model' is not defined

In [6]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-cased')

import torch
label_all_tokens = False

def align_label(texts, labels):
    tokenized_inputs = tokenizer(texts, padding='max_length', max_length=120, truncation=True)

    word_ids = tokenized_inputs.word_ids()

    previous_word_idx = None
    label_ids = []

    for word_idx in word_ids:

        if word_idx is None:
            label_ids.append(-100)

        elif word_idx != previous_word_idx:
            try:
                label_ids.append(labels_to_ids[labels[word_idx]])
            except:
                label_ids.append(-100)
        else:
            try:
                label_ids.append(labels_to_ids[labels[word_idx]] if label_all_tokens else -100)
            except:
                label_ids.append(-100)
        previous_word_idx = word_idx

    return label_ids

class DataSequence(torch.utils.data.Dataset):

    def __init__(self, df):

        lb = [i.split() for i in df['labels'].values.tolist()]
        txt = df['text'].values.tolist()
        self.texts = [tokenizer(str(i),
                               padding='max_length', max_length = 120, truncation=True, return_tensors="pt") for i in txt]
        self.labels = [align_label(i,j) for i,j in zip(txt, lb)]

    def __len__(self):

        return len(self.labels)

    def get_batch_data(self, idx):

        return self.texts[idx]

    def get_batch_labels(self, idx):

        return torch.LongTensor(self.labels[idx])

    def __getitem__(self, idx):

        batch_data = self.get_batch_data(idx)
        batch_labels = self.get_batch_labels(idx)

        return batch_data, batch_labels

In [7]:
import numpy as np

# Set the random seed for reproducibility
np.random.seed(42)

# Shuffle the DataFrame rows
df = df.sample(frac=1, random_state=42)

# Calculate the indices to split the DataFrame
train_size = int(0.9 * len(df))
val_size = len(df) - train_size

# Split the DataFrame into df_train and df_val
df_train = df[:train_size]
df_val = df[train_size:]

In [8]:
class BertModel(torch.nn.Module):

    def __init__(self):

        super(BertModel, self).__init__()

        self.bert = BertForTokenClassification.from_pretrained('bert-base-cased', num_labels=len(sorted_unique_labels))

    def forward(self, input_id, mask, label):

        output = self.bert(input_ids=input_id, attention_mask=mask, labels=label, return_dict=False)

        return output

In [9]:
def train_loop(model, df_train, df_val):

    train_dataset = DataSequence(df_train)
    val_dataset = DataSequence(df_val)

    train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    optimizer = SGD(model.parameters(), lr=LEARNING_RATE)

    if use_cuda:
        model = model.cuda()

    best_acc = 0
    best_loss = 1000

    for epoch_num in range(EPOCHS):

        total_acc_train = 0
        total_loss_train = 0
        count = 0

        #model.train()

        for train_data, train_label in tqdm(train_dataloader):
            count = count + 1

            train_label = train_label.to(device)
            mask = train_data['attention_mask'].squeeze(1).to(device)
            input_id = train_data['input_ids'].squeeze(1).to(device)

            optimizer.zero_grad()
            loss, logits = model(input_id, mask, train_label)

            for i in range(logits.shape[0]):

              logits_clean = logits[i][train_label[i] != -100]
              label_clean = train_label[i][train_label[i] != -100]

              predictions = logits_clean.argmax(dim=1)
              acc = (predictions == label_clean).float().mean()
              total_acc_train += acc
              total_loss_train += loss.item()
              
            loss.backward()
            optimizer.step()

        model.eval()

        total_acc_val = 0
        total_loss_val = 0

        for val_data, val_label in val_dataloader:

            val_label = val_label.to(device)
            mask = val_data['attention_mask'].squeeze(1).to(device)
            input_id = val_data['input_ids'].squeeze(1).to(device)

            loss, logits = model(input_id, mask, val_label)

            for i in range(logits.shape[0]):

              logits_clean = logits[i][val_label[i] != -100]
              label_clean = val_label[i][val_label[i] != -100]

              predictions = logits_clean.argmax(dim=1)
              acc = (predictions == label_clean).float().mean()
              total_acc_val += acc
              total_loss_val += loss.item()

        print(
            f'Epochs: {epoch_num + 1} | Loss: {total_loss_train / len(df_train): .3f} | Accuracy: {total_acc_train: .3f} | Val_Loss: {total_loss_val / len(df_val): .3f} | Accuracy: {total_acc_val / len(df_val): .3f}')
LEARNING_RATE = 5e-3
EPOCHS = 5
BATCH_SIZE = 4
model = BertModel()
train_loop(model, df_train, df_val)


Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cas

23
23
23
23
23


In [13]:
# Save the trained model
model_path = './outputs/model21.pth'


# Save the model using save_pretrained
model.bert.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)


('./outputs/model21.pth/tokenizer_config.json',
 './outputs/model21.pth/special_tokens_map.json',
 './outputs/model21.pth/vocab.txt',
 './outputs/model21.pth/added_tokens.json',
 './outputs/model21.pth/tokenizer.json')

In [1]:
model_path = './outputs/model21.pth'
# Load the model using from_pretrained
model1 = BertForTokenClassification.from_pretrained(model_path).to("cuda")
#model = BertForSequenceClassification.from_pretrained(model_path, num_labels=len(target_names))
tokenizer = BertTokenizerFast.from_pretrained(model_path)

NameError: name 'BertForTokenClassification' is not defined

In [1]:
# import required libraries
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential

from azure.ai.ml import MLClient, Input
from azure.ai.ml.dsl import pipeline
from azure.ai.ml import load_component

In [2]:
# Handle to the workspace
from azure.ai.ml import MLClient

# Authentication package
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

# Get a handle to the workspace
ml_client = MLClient(
    credential=credential,
    subscription_id="",
    resource_group_name="",
    workspace_name="",
)

In [25]:
import urllib.request
import json
import os
import ssl

def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

allowSelfSignedHttps(True) # this line is needed if you use self-signed certificate in your scoring service.

# Request data goes here
# The example below assumes JSON formatting which may be updated
# depending on the format your endpoint expects.
# More information can be found here:
# https://docs.microsoft.com/azure/machine-learning/how-to-deploy-advanced-entry-script
data = {
	"data": [
		{
			"Title": "Gaok Men's Retro Cotton Multi-Pocket Camo Cargo Shorts",
			"Description": "Random Description",
			"Price": "13",
			"Category": "Jeany Jeans",
			"Image": "https://www.ssxsds.de/test.jpg"
		},
		{
			"Title": "Honchosfx Mens Retro Creedence Clearwater Revival 1971 T-Shirt Brown",
			"Description": "ssdsdsdssdssd",
			"Price": "15",
			"Category": "random",
			"Image": "https://www.ssxsds.de/test.jpg"
		}
	]
}	

body = str.encode(json.dumps(data))

url = ''
# Replace this with the primary/secondary key or AMLToken for the endpoint
api_key = ''
if not api_key:
    raise Exception("A key should be provided to invoke the endpoint")

# The azureml-model-deployment header will force the request to go to a specific deployment.
# Remove this header to have the request observe the endpoint traffic rules
headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key), 'azureml-model-deployment': 'uc1' }

req = urllib.request.Request(url, body, headers)
try:
    response = urllib.request.urlopen(req)

    result = response.read()
    print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))

    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(error.read().decode("utf8", 'ignore'))

b'{"data": [{"Title": "Gaok Men\'s Retro Cotton Multi-Pocket Camo Cargo Shorts", "Description": "Random Description", "Price": "13", "Category": "Jeany Jeans", "Image": "https://www.ssxsds.de/test.jpg"}, {"Title": "Honchosfx Mens Retro Creedence Clearwater Revival 1971 T-Shirt Brown", "Description": "ssdsdsdssdssd", "Price": "15", "Category": "random", "Image": "https://www.ssxsds.de/test.jpg"}]}'
b'{"status": "success", "result": [{"general": [{"types": ["Dog supplies"], "all_attributes": [{"Title": "string", "Description": "string", "Price": "decimal", "Category": "string", "Type": "string", "Image": "file"}], "common_attributes": [{"Title": "string", "Description": "string", "Price": "file", "Category": "string"}]}]}], "data": [{"Title": [{"input": "Gaok Men\'s Retro Cotton Multi-Pocket Camo Cargo Shorts", "status": "mapped", "tags": [{"Gaok": [{"attribute": "BRAND"}]}, {"Camo Cargo Shorts": [{"attribute": "PATTERN"}]}]}], "Description": [{"input": "Random Description", "status": "u

In [3]:
from azure.ai.ml.entities import Environment

custom_env_name = "ml-use-case-1"

pipeline_job_env = Environment(
    name=custom_env_name,
    version="19",
)


print(
    f"Environment with name {pipeline_job_env.name} is registered to workspace, the environment version is {pipeline_job_env.version}"
)

Environment with name ml-use-case-1 is registered to workspace, the environment version is 19


In [6]:
from azure.ai.ml import command
from azure.ai.ml.entities import Data
from azure.ai.ml import Input
from azure.ai.ml import Output
from azure.ai.ml.constants import AssetTypes

# === Note on path ===
# can be can be a local path or a cloud path. AzureML supports https://`, `abfss://`, `wasbs://` and `azureml://` URIs.
# Local paths are automatically uploaded to the default datastore in the cloud.
# More details on supported paths: https://docs.microsoft.com/azure/machine-learning/how-to-read-write-data-v2#supported-paths

inputs = {
    "data_dir": Input(
        type=AssetTypes.URI_FILE, path=''
    ),  # path="azureml:azureml_stoic_cartoon_wgb3lgvgky_output_data_cifar:1"), #path="azureml://datastores/workspaceblobstore/paths/azureml/stoic_cartoon_wgb3lgvgky/cifar/"),
    "epochs":50,
    "batch_size": 16,
    "learning_rate": 2e-3 ,
}

job = command(
    code="./",  # local path where the code is stored
    command="python demo.py --data_dir ${{inputs.data_dir}} --epochs ${{inputs.epochs}} --batch_size ${{inputs.batch_size}} --learning_rate ${{inputs.learning_rate}}",
    inputs=inputs,
    environment=f"{pipeline_job_env.name}:{pipeline_job_env.version}",
    compute="gpu",  # Change the name to the gpu cluster of your workspace.
    instance_count=1,  # In this, only 2 node cluster was created.
    distribution={
        "type": "PyTorch",
        # set process count to the number of gpus per node
        # NV6 has only 1 GPU
        "process_count_per_instance": 4,
    },
)

In [7]:
ml_client.jobs.create_or_update(job)

Your file exceeds 100 MB. If you experience low speeds, latency, or broken connections, we recommend using the AzCopyv10 tool for this file transfer.

Example: azcopy copy '/mnt/batch/tasks/shared/LS_root/mounts/clusters/safira-cpu/code/Users/r.mandal/UC1_bio' 'https://safiraml9781015086.blob.core.windows.net/584b2160-4-8b11e43e-0492-5dd2-aef8-5b439a05f82c/UC1_bio' 

See https://docs.microsoft.com/azure/storage/common/storage-use-azcopy-v10 for more information.
Uploading UC1_bio (141.22 MBs): 100%|██████████| 141223221/141223221 [00:01<00:00, 119749718.43it/s]




Experiment,Name,Type,Status,Details Page
UC1_bio,olive_ticket_d3jylgfzrb,command,Starting,Link to Azure Machine Learning studio


In [12]:
# we will reuse the command_job created before. we call it as a function so that we can apply inputs
# we do not apply the 'iris_csv' input again -- we will just use what was already defined earlier
from azure.ai.ml.sweep import Choice, Uniform, MedianStoppingPolicy,LogUniform
command_job_for_sweep = job(
    learning_rate= LogUniform(2e-3, 5e-5),
    batch_size=Choice(values=[2, 4, 8, 16]),
)

# apply the sweep parameter to obtain the sweep_job
sweep_job = command_job_for_sweep.sweep(
    compute="gpu",  # Change the name to the gpu cluster of your workspace.
    sampling_algorithm="random",
    primary_metric="validation_accuracy",
    goal="Maximize",
    max_total_trials=8,
    max_concurrent_trials=4,
   early_termination_policy = MedianStoppingPolicy(delay_evaluation = 10, evaluation_interval = 1),
)

In [14]:
# submit the sweep job
returned_sweep_job = ml_client.jobs.create_or_update(sweep_job)

# stream the output and wait until the job is finished
ml_client.jobs.stream(returned_sweep_job.name)

# refresh the latest status of the job after streaming
returned_sweep_job = ml_client.jobs.get(name=returned_sweep_job.name)

# get a URL for the status of the job
returned_sweep_job.studio_url

Your file exceeds 100 MB. If you experience low speeds, latency, or broken connections, we recommend using the AzCopyv10 tool for this file transfer.

Example: azcopy copy '/mnt/batch/tasks/shared/LS_root/mounts/clusters/safira-cpu/code/Users/r.mandal/UC1_bio' 'https://safiraml9781015086.blob.core.windows.net/584b2160-4-3aae912c-0db5-5ff9-b522-1452554edc10/UC1_bio' 

See https://docs.microsoft.com/azure/storage/common/storage-use-azcopy-v10 for more information.
Uploading UC1_bio (141.2 MBs): 100%|██████████| 141197539/141197539 [00:01<00:00, 119764380.99it/s]




HttpResponseError: (UserError) Validation failed.
 Error : Expected 5E-05 to be greater than or equal to 0.002

Code: UserError
Message: Validation failed.
 Error : Expected 5E-05 to be greater than or equal to 0.002

Additional Information:Type: ComponentName
Info: {
    "value": "managementfrontend"
}Type: Correlation
Info: {
    "value": {
        "operation": "0f3025731cffb859f9fd59f14f35dd4b",
        "request": "f354cf524a5db642"
    }
}Type: Environment
Info: {
    "value": "eastus"
}Type: Location
Info: {
    "value": "eastus"
}Type: Time
Info: {
    "value": "2023-07-19T04:29:43.7938891+00:00"
}

In [17]:
# Handle to the workspace
from azure.ai.ml import MLClient

# Authentication package
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

In [2]:
# Get a handle to the workspace
ml_client = MLClient(
    credential=credential,
    subscription_id="",
    resource_group_name="",
    workspace_name="",
)

In [3]:
import uuid

# Creating a unique name for the endpoint
online_endpoint_name = "uc1-" + str(uuid.uuid4())[:8]

In [4]:
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
)

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="this is an online endpoint for uc1",
    auth_mode="key",
    tags={
        "training_dataset": "product dataste",
        "model_type": "OpenTag",
    },
)

endpoint_result = ml_client.begin_create_or_update(endpoint).result()

print(
    f"Endpint {endpoint_result.name} provisioning state: {endpoint_result.provisioning_state}"
)

Endpint uc1-37e94cda provisioning state: Succeeded


In [5]:
endpoint = ml_client.online_endpoints.get(name=online_endpoint_name)

print(
    f'Endpint "{endpoint.name}" with provisioning state "{endpoint.provisioning_state}" is retrieved'
)

Endpint "uc1-37e94cda" with provisioning state "Succeeded" is retrieved


In [6]:
# Let's pick the latest version of the model
latest_model_version = max(
    [int(m.version) for m in ml_client.models.list(name="uc1_token")]
)

In [22]:
import os
env = Environment(
    conda_file=os.path.join("./dependencies", "conda.yaml"),
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest",
)

In [23]:
from azure.ai.ml import MLClient
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    CodeConfiguration,
    Environment,
)
from azure.identity import DefaultAzureCredential

# picking the model to deploy. Here we use the latest version of our registered model
model = ml_client.models.get(name="uc1_token", version=latest_model_version)


# create an online deployment.
red_deployment_uc1 = ManagedOnlineDeployment(
    name="uc1",
    endpoint_name=online_endpoint_name,
    model=model,
    code_configuration=CodeConfiguration(
        code="./",
        scoring_script="score.py"
    ),
    environment=env,
    instance_type="Standard_E2s_v3",
    instance_count=1,
)

red_deployment_results = ml_client.online_deployments.begin_create_or_update(
    red_deployment_uc1
).result()

print(
    f"Deployment {red_deployment_results.name} provisioning state: {red_deployment_results.provisioning_state}"
)

NameError: name 'latest_model_version' is not defined

In [19]:
ml_client.online_endpoints.invoke(
    endpoint_name="uc1-37e94cda",
    deployment_name="uc1",
    request_file=os.path.join("./dependencies", "sam.json"),
)

'{"status": "success", "result": [{"general": [{"types": ["Dog supplies"], "all_attributes": [{"Title": "string", "Description": "string", "Price": "decimal", "Category": "string", "Type": "string", "Image": "file"}], "common_attributes": [{"Title": "string", "Description": "string", "Price": "file", "Category": "string"}]}]}], "data": [{"Title": [{"input": "Gaok Men\'s Retro Cotton Multi-Pocket Camo Cargo Shorts", "status": "mapped", "tags": [{"Gaok": [{"attribute": "BRAND"}]}, {"Camo Cargo Shorts": [{"attribute": "PATTERN"}]}]}], "Description": [{"input": "Random Description", "status": "unmapped"}], "Price": [{"input": "13", "status": "unmapped"}], "Category": [{"input": "Jeany Jeans", "status": "mapped", "tags": {"Jeany Jeans": [{"attribute": "category", "probability": "100%"}]}}], "Image": [{"input": "https://www.ssxsds.de/test.jpg", "status": "unmapped"}]}, {"Title": [{"input": "Honchosfx Mens Retro Creedence Clearwater Revival 1971 T-Shirt Brown", "status": "mapped", "tags": [{"